In [21]:
Pkg.add("ProgressMeter")

INFO: Package ProgressMeter is already installed
INFO: METADATA is out-of-date — you may not have the latest version of ProgressMeter
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [23]:
using Flux
using Flux: onehot, argmax, chunk, batchseq, throttle, crossentropy
using StatsBase: wsample
using Base.Iterators: partition
using CuArrays
using ProgressMeter

In [3]:
# Download the dataset
isfile("input.txt") ||
  download("http://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt",
           "input.txt")

true

In [4]:
text = collect(readstring("input.txt"))
alphabet = [unique(text)..., '_'];

In [6]:
text = map(ch -> onehot(ch, alphabet), text);

In [7]:
stop = onehot('_', alphabet);

In [8]:
N = length(alphabet)
seqlen = 50
nbatch = 50

50

In [9]:
X_train = collect(partition(batchseq(chunk(text, nbatch), stop), seqlen))
y_train = collect(partition(batchseq(chunk(text[2:end], nbatch), stop), seqlen))

1830-element Array{Array{Flux.OneHotMatrix{Array{Flux.OneHotVector,1}},1},1}:
 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}[Bool[false false … false false; true false … false false; … ; false false … false false; false false … false false], Bool[false false … false false; false false … false false; … ; false false … false false; false false … false false], Bool[false false … false false; false false … false false; … ; false false … false false; false false … false false], Bool[false false … false false; false false … false false; … ; false false … false false; false false … false false], Bool[false false … false false; false false … false false; … ; false false … false false; false false … false false], Bool[false false … false false; false false … false false; … ; false false … false false; false false … false false], Bool[false false … false false; true false … false false; … ; false false … false false; false false … false false], Bool[false false … false false; false false … false

In [115]:
model = Chain(
  LSTM(N, 128),
  LSTM(128, 128),
  Dense(128, N),
  softmax)

model = mapleaves(cu, model)

Chain(Recur(LSTMCell(68, 128)), Recur(LSTMCell(128, 128)), Dense(128, 68), NNlib.softmax)

In [71]:
function loss(xs, ys)
    l = sum(crossentropy.(model.(xs), ys))
    Flux.truncate!(model)
    return l
end

loss (generic function with 1 method)

In [72]:
opt = ADAM(params(model), 0.01)

(::#71) (generic function with 1 method)

In [73]:
evalcb = () -> @show loss(X_train[5], y_train[5])

(::#31) (generic function with 1 method)

In [ ]:
Flux.train!(loss, zip(X_train, y_train), opt,
            cb = throttle(evalcb, 30))

In [76]:
batch_num = length(y_train)

@showprogress 1 "Training..." for idx in 1:batch_num
    batch_x, batch_y = CuArray.(X_train[idx]), CuArray.(y_train[idx])
#     println(typeof(batch_x))
    println("Batch $(idx)")
    Flux.train!(loss, [(batch_x, batch_y)], opt)
end

Batch 1


LoadError: [91mArrays type contains pointer - this doesn't work on the GPU (So arrays that don't contain pointers to memory, e.g. UnitRanges, would work).
            Try converting it to a gpu array. Found: BitArray{2}[39m

In [98]:
model.(cu(X_train[1]))

LoadError: [91mBroadcast output type Any is not concrete[39m

In [104]:
(X_train[1][1])

68×50 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
  true  false  false  false  false  …  false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false   true  false
 false  false  false  false   true     false   true   true  false  false
 false  false  false  false  false      true  false  false  false  false
 false  false  false  false  false  …  false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false   true  false  false  false     false  false  false  false   true
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false  …  false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  fals

In [118]:
model(cu(X_train[1][1]))

LoadError: [91mArrays type contains pointer - this doesn't work on the GPU (So arrays that don't contain pointers to memory, e.g. UnitRanges, would work).
            Try converting it to a gpu array. Found: BitArray{2}[39m